## Description:

Gets and stores data for nodes (representing street segments)

In [ ]:
import osmnx as ox
import folium
from geopy.distance import geodesic
from shapely.geometry import Polygon


def filter_edges_by_grid(edges, grid_square):
    print("filtering edges")
    west, south, east, north = grid_square
    grid_polygon = Polygon([(west, south), (west, north), (east, north), (east, south)])
    
    # Use the GeoDataFrame's spatial methods to check if edge geometries intersect with the grid square
    filtered_edges = edges[edges.geometry.apply(lambda geom: geom.intersects(grid_polygon))]
    return filtered_edges


# Function to interpolate points every 10 feet
def interpolate_points(start, end, step_ft):
    points = [start]
    total_distance = geodesic(start, end).feet
    step_ratio = step_ft / total_distance
    while step_ratio < 1.0:
        interp_lat = start[0] + (end[0] - start[0]) * step_ratio
        interp_lon = start[1] + (end[1] - start[1]) * step_ratio
        points.append((interp_lat, interp_lon))
        step_ratio += step_ft / total_distance
    points.append(end)  # Ensure the end point is included
    return points

def create_map_for_edges(edges):
    global grid_counter

    FEET_SPACING
    grid_counter += 1
    print("Creating map for Edges")
    point_counter = 0

    # Initialize a Folium map
    m = folium.Map(location=center_point, zoom_start=16)
    
    # Add edges to the map
    for _, row in edges.iterrows():
        coords = [(y, x) for x, y in row['geometry'].coords]
        
        # Interpolate points between each pair of coordinates
        for i in range(len(coords) - 1):
            start = coords[i]
            end = coords[i + 1]
            interpolated_points = interpolate_points(start, end, step_ft=FEET_SPACING)
            
            # Plot each interpolated point
            for point in interpolated_points:
                point_counter += 1
                # folium.Marker(location=point, popup=f"({location[0]}, {location[1]})").add_to(mymap)
                folium.CircleMarker(
                    location=point,
                    radius=2,
                    color='red',
                    fill=True,
                    fill_opacity=0.8,
                    popup=f"({point[0]}, {point[1]})"
                ).add_to(m)
    
    # Save the map
    m.save(f"data/dc_nodes/dc_{grid_counter}.html")
    print(f"Map created with {point_counter} points.")


def main():

    print("Starting")

    graph = ox.graph_from_place("Washington, DC, USA", network_type='drive')

    print("getting nodes and edges")
    nodes, edges = ox.graph_to_gdfs(graph)

    # prep for batching (since these is way to large of a dataset)
    bbox = edges.total_bounds  # [west, south, east, north]
    print("got bounds")


    # Divide the bounding box into 20 grids (4x5)
    num_rows = 6
    num_cols = 7
    grid_width = (bbox[2] - bbox[0]) / num_cols  # (east - west) / cols
    grid_height = (bbox[3] - bbox[1]) / num_rows  # (north - south) / rows

    grid_squares = []
    print("creating square")
    for row in range(num_rows):
        for col in range(num_cols):
            west = bbox[0] + col * grid_width
            east = west + grid_width
            south = bbox[1] + row * grid_height
            north = south + grid_height
            grid_squares.append((west, south, east, north))

    print(f"grid squares: {len(grid_squares)}")
    for idx, grid_square in enumerate(grid_squares):
        filtered_edges = filter_edges_by_grid(edges, grid_square)
        create_map_for_edges(filtered_edges)

grid_counter = 0
FEET_SPACING = 100
center_point = (38.90251234580012, -77.04350506490759)


main()


In [ ]:
import osmnx as ox
point = (38.90373,-77.0488552)
# graph = ox.graph_from_place("Washington, DC, USA", network_type='drive')
graph = ox.graph_from_point(point,400, network_type='drive')

nodes, edges = ox.graph_to_gdfs(graph)





In [ ]:
import folium

# Create a folium map centered around the average latitude and longitude of the nodes
center_lat = nodes.geometry.y.mean()
center_lon = nodes.geometry.x.mean()
dc_map = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Add a marker for each node
for node_id, node in nodes.iterrows():
    print(node)
    lat = node.geometry.y
    lon = node.geometry.x
    # folium.Marker(location=[lat, lon], popup=f"Node ID: {node_id}").add_to(dc_map)
    folium.CircleMarker(
                    location=[lat, lon],
                    radius=2,
                    color='red',
                    fill=True,
                    fill_opacity=0.8,
                    popup=f"({point[0]}, {point[1]})"
                ).add_to(dc_map)


# Add edges to the map with different colors for one-way and two-way edges
for _, edge in edges.iterrows():
    # Extract edge geometry
    if edge.geometry.geom_type == 'LineString':             
        coords = [(lat, lon) for lon, lat in edge.geometry.coords]
        color = 'red' if edge.get('oneway', False) else 'blue'
        folium.PolyLine(coords, color=color, weight=2.5, opacity=0.8).add_to(dc_map)
    elif edge.geometry.geom_type == 'MultiLineString':
        for line in edge.geometry:
            coords = [(lat, lon) for lon, lat in line.coords]
            color = 'red' if edge.get('oneway', False) else 'blue'
            folium.PolyLine(coords, color=color, weight=2.5, opacity=0.8).add_to(dc_map)

# Save or display the map
dc_map.save("data/test_washington_dc_nodes_map.html")
print("Saved")

# print(nodes)
# print(edges)

In [39]:
def draw_segments_on_map(segments):
    m = folium.Map(location=[center_lat, center_lon], zoom_start=12)
    for segment_id, segment in segments.items():
        lat = segment['lat']
        long = segment['long']
        folium.CircleMarker(
            location=[lat, long],
            radius=2,
            color='red',
            fill=True,
            fill_opacity=0.8,
            popup=f"({segment['headings']})"
        ).add_to(m)

    m.save("data/dc_segments_map.html")

In [40]:
import math
import pandas as pd

def write_as_csv(filepath, dict):
    df = pd.DataFrame.from_dict(dict, orient='index')
    df.to_csv(filepath)

def segment_key(lat, long):
    return f"{lat}_{long}"


def calculate_heading(lat1, lon1, lat2, lon2):
    """
    Calculate the heading (bearing) from one point to another.
    Returns the heading in degrees.
    """
    d_lon = math.radians(lon2 - lon1)
    lat1 = math.radians(lat1)
    lat2 = math.radians(lat2)

    x = math.sin(d_lon) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - (math.sin(lat1) * math.cos(lat2) * math.cos(d_lon))
    initial_heading = math.atan2(x, y)
    # Convert to degrees and normalize to 0-360
    heading = (math.degrees(initial_heading) + 360) % 360
    return heading

def grab_store_all_segments(nodes, edges):
    segments = {} #key = lat_long, value = lat, long, headings[], segment_links[]

    # first loop through all nodes and add them as segments
    for node_id, node in nodes.iterrows():
        lat = node.geometry.y
        long = node.geometry.x

        print(node_id)
        print(f"Node:{segment_key(lat,long)}")
        segment_headings = []
        segment_links = []

        successors = list(graph.successors(node_id))
        print(successors)

        # loop through all edges to find the ones that are connected to the node
        for successor_id in successors:
            successor_node = nodes.loc[successor_id]  # Access the node data from the GeoDataFrame
            successor_lat = successor_node.geometry.y
            successor_long = successor_node.geometry.x
            print(f"  Successor Node ID: {successor_id}, Latitude: {successor_lat}, Longitude: {successor_long}")
            
            # # add segment link
            segment_links.append(segment_key(successor_lat, successor_long))

            # # calculate segment heading
            heading = calculate_heading(lat, long, successor_lat, successor_long)
            segment_headings.append(heading)


        segments[segment_key(lat,long)] = {
            "lat": lat,
            "long": long, 
            "headings": segment_headings, 
            "segment_links": segment_links}
    
    print("PRINTING SEGMENTS:")
    print(segments)
    write_as_csv("data/dc_segments.csv", segments)
    draw_segments_on_map(segments)


grab_store_all_segments(nodes, edges)


49717477
Node:38.9023974_-77.0466515
[641116257, 1381210497]
  Successor Node ID: 641116257, Latitude: 38.9023963, Longitude: -77.0458805
  Successor Node ID: 1381210497, Latitude: 38.9013267, Longitude: -77.0466507
49729677
Node:38.9027349_-77.0494689
[1645991534]
  Successor Node ID: 1645991534, Latitude: 38.9029695, Longitude: -77.0497107
49732327
Node:38.9026491_-77.051416
[49737722, 632935663, 49787770]
  Successor Node ID: 49737722, Latitude: 38.9024153, Longitude: -77.051421
  Successor Node ID: 632935663, Latitude: 38.9026486, Longitude: -77.0530701
  Successor Node ID: 49787770, Latitude: 38.9029901, Longitude: -77.0514153
49737722
Node:38.9024153_-77.051421
[49732327, 49737731, 49765234]
  Successor Node ID: 49732327, Latitude: 38.9026491, Longitude: -77.051416
  Successor Node ID: 49737731, Latitude: 38.9023368, Longitude: -77.050763
  Successor Node ID: 49765234, Latitude: 38.9011257, Longitude: -77.0514282
49737731
Node:38.9023368_-77.050763
[5521303041]
  Successor Node I

[segmentid] = lat, long, headings[], segment_links[], 

segment_links -> which other segment ids can you get to from this one.
for example, on a one way street, you can only get to the next one, but on a two way, you can get to the previous and next one...

Note, some segment links may not have a respective google maps tile (and thats okay), we still want to keep track of links in case we want to calculate pathing

make each intersection its own segment